In [ ]:
!pip install datasets==2.10.1
!pip install transformers==4.27.3
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install audiomentations

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from huggingface_hub import login
from datasets import interleave_datasets
import evaluate
import numpy as np
from datasets import Audio
import torch
from audiomentations import *

In [ ]:
login(token='')

In [ ]:
model=''
tokenizer='openai/whisper-small'
pipe = pipeline('automatic-speech-recognition', device=torch.device(0), model=model, tokenizer=tokenizer)

In [ ]:
yt_clean = load_dataset('aanosov/youtube_control_001', split='test', use_auth_token=True)
yt_aug = load_dataset('aanosov/youtube_control_001_aug', split='train', use_auth_token=True)

golos_clean = load_dataset('aanosov/golos_test', split='test', use_auth_token=True)
golos_aug = load_dataset('aanosov/golos_test_aug', split='test', use_auth_token=True)

calls_clean = load_dataset('aanosov/asr_calls', split='test', use_auth_token=True)
calls_aug = load_dataset('aanosov/asr_calls_aug', split='test', use_auth_token=True)

cv_clean = load_dataset('aanosov/common_voice_ru_test', split='test', use_auth_token=True)
cv_aug = load_dataset('aanosov/common_voice_ru_test_aug', split='test', use_auth_token=True)

sova_clean = load_dataset('aanosov/mfdp_sova_part2', split='test', use_auth_token=True)
sova_aug = load_dataset('aanosov/mfdp_sova_part2_aug', split='test', use_auth_token=True)

In [ ]:
import re
def normalize(x):
    x = [re.sub("[^0-9A-Za-zА-Яёа-яЁ ]", " ", t.lower()) for t in x]
    x = [' '.join(t.split()) for t in x]
    return x

In [ ]:
i=0
for dtst in [yt_clean,yt_aug,  golos_clean, golos_aug, calls_clean, calls_aug, cv_clean, cv_aug, sova_clean, sova_aug]:
    
    i+=1 
    
    key_words = []
    predictions = []
    answers = []

    for example in dtst:
        if (len(list(example['answer'].split()))!=0) and len(example['audio']['array'])>0:
            try:
                key_words.append(example['key_words'])
            except:
                pass
            answers.append(example['answer'])
            result = pipe(example['audio']['array'], 
                          chunk_length_s=30, 
                          generate_kwargs={"language": "<|ru|>", "task": "transcribe"})['text']
            result_list = result.split()
            q = 0
            while q < (len(result_list)-2):
                if (result_list[q]==result_list[q+1]) and (result_list[q]==result_list[q+2]):
                    del result_list[q+2]
                else:
                    q+=1
            result = ' '.join(result_list)
            predictions.append(result)
            
    wer_calculator = evaluate.load("wer")
    cer_calculator = evaluate.load("cer")

    wer = wer_calculator.compute(predictions=normalize(predictions), references=normalize(answers))
    cer = cer_calculator.compute(predictions=normalize(predictions), references=normalize(answers))

    print(i)
    print(f"wer: {wer}")
    print(f"cer: {cer}") 

    if len(key_words)>0:
        m = 0
        n = 0

        predictions = normalize(predictions)
        key_words = ['none' if v is None else v for v in key_words]
        key_words = normalize(key_words)

        for sample in range(len(predictions)):
            pred = predictions[sample]
            for key_word in key_words[sample]:
                if (key_word!='none') and (key_word in pred):
                    pred = pred.replace(key_word, '', 1)
                    m+=1
                n+=1

        print('key_words_rate:', m/n)
    print()